In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ipywidgets import interact
import requests
from PIL import Image
import io
%matplotlib inline

In [2]:
def kuwahara(pic,r=5,resize=False,rate=0.5):
    h,w,_=pic.shape
    if resize:pic=cv2.resize(pic,(int(w*rate),int(h*rate)));h,w,_=pic.shape
    pic=np.pad(pic,((r,r),(r,r),(0,0)),"edge")
    ave,var=cv2.integral2(pic)
    ave=((ave[:-r-1,:-r-1]+ave[r+1:,r+1:]-ave[r+1:,:-r-1]-ave[:-r-1,r+1:])/(r+1)**2)
    var=((var[:-r-1,:-r-1]+var[r+1:,r+1:]-var[r+1:,:-r-1]-var[:-r-1,r+1:])/(r+1)**2-ave**2).sum(axis=2)
    def filt(i,j):
        return np.array([ave[i,j],ave[i+r,j],ave[i,j+r],ave[i+r,j+r]])[(np.array([var[i,j],var[i+r,j],var[i,j+r],var[i+r,j+r]]).argmin(axis=0).flatten(),j.flatten(),i.flatten())].reshape(w,h,_).transpose(1,0,2)
    filtered_pic = filt(*np.meshgrid(np.arange(h),np.arange(w))).astype(pic.dtype)
    return filtered_pic

In [3]:
url = input("URL:")

pic = np.array(Image.open(io.BytesIO(requests.get(url).content)))

@interact(r=(1,30),rate=(0,150,10))
def interactkuwa(r,rate):
    im=pic.copy()
    plt.figure(figsize=(15, 15))
    filtered_pic=kuwahara(im,r,1,rate/100)
    plt.imshow(filtered_pic)

URL:https://camo.qiitausercontent.com/bfe670fb577994f0a3858a97bfb7bfa8d85eb201/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f3530313238312f37643331353937302d376432352d333865342d303866622d3934323263396332646239342e6a706567


interactive(children=(IntSlider(value=15, description='r', max=30, min=1), IntSlider(value=70, description='ra…